# Databricks notebook: Silver - FireRisk Diário (INPE)

In [0]:
# =========================================================
# Converte dados NetCDF (Bronze) em tabela Silver padronizada
# =========================================================

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit, to_timestamp
spark = SparkSession.builder.getOrCreate()

In [0]:
# =========================================================
# 🔹 Parâmetros do Job
# =========================================================
dbutils.widgets.text("catalog", "")
dbutils.widgets.text("schema_bronze", "")
dbutils.widgets.text("table_bronze", "")
dbutils.widgets.text("schema_silver", "")
dbutils.widgets.text("table_silver", "")
dbutils.widgets.text("data_ref_carga", "")

catalog         = dbutils.widgets.get("catalog")
schema_bronze   = dbutils.widgets.get("schema_bronze")
table_bronze    = dbutils.widgets.get("table_bronze")
schema_silver   = dbutils.widgets.get("schema_silver")
table_silver    = dbutils.widgets.get("table_silver")
data_ref_carga  = dbutils.widgets.get("data_ref_carga")

if not data_ref_carga:
    raise ValueError("❌ Parâmetro 'data_ref_carga' não informado")

print("=====================================")
print(f"📅 Data de referência : {data_ref_carga}")
print(f"📦 Bronze origem      : {catalog}.{schema_bronze}.{table_bronze}")
print(f"💾 Silver destino     : {catalog}.{schema_silver}.{table_silver}")
print("=====================================")

In [0]:
# =========================================================
# 🔹 Leitura da Bronze
# =========================================================
df_bronze = spark.table(f"{catalog}.{schema_bronze}.{table_bronze}") \
    .filter(col("data_ref_carga") == data_ref_carga)

print(f"🔸 Linhas lidas: {df_bronze.count()}")

In [0]:
# =========================================================
# 🔹 Limpeza e enriquecimento
# =========================================================
df_silver = (
    df_bronze
    .filter(col("rf").isNotNull())  # remove pontos sem valor
    .withColumn("timestamp", to_timestamp(col("time")))  # padroniza tempo
    .withColumn("nivel_risco",
        when(col("rf") < 0.2, "baixo")
        .when(col("rf") < 0.4, "moderado")
        .when(col("rf") < 0.6, "alto")
        .when(col("rf") < 0.8, "muito alto")
        .otherwise("extremo")
    )
    .withColumn("data_ref_carga", lit(data_ref_carga))
)

print(f"✅ Linhas após limpeza: {df_silver.count()}")

In [0]:
# =========================================================
# 🔹 Escrita na Tabela Silver (Delta)
# =========================================================
(
    df_silver.write
    .format("delta")
    .mode("overwrite")
    .option("replaceWhere", f"data_ref_carga = '{data_ref_carga}'")
    .partitionBy("data_ref_carga")
    .saveAsTable(f"{catalog}.{schema_silver}.{table_silver}")
)

print(f"💾 Dados gravados em: {catalog}.{schema_silver}.{table_silver}")

In [0]:
# =========================================================
# 🔹 Adiciona descrições (comments) na tabela Silver
# =========================================================
spark.sql(f"""
COMMENT ON TABLE {catalog}.{schema_silver}.{table_silver} IS
'Tabela Silver do modelo INPE Fire Risk 2.2. Representa o risco diário de fogo em grade (latitude, longitude).';

COMMENT ON COLUMN {catalog}.{schema_silver}.{table_silver}.timestamp IS 'Data e hora da modelagem meteorológica';
COMMENT ON COLUMN {catalog}.{schema_silver}.{table_silver}.lat IS 'Latitude em graus decimais';
COMMENT ON COLUMN {catalog}.{schema_silver}.{table_silver}.lon IS 'Longitude em graus decimais';
COMMENT ON COLUMN {catalog}.{schema_silver}.{table_silver}.rf IS 'Índice de risco de fogo (0 a 1)';
COMMENT ON COLUMN {catalog}.{schema_silver}.{table_silver}.nivel_risco IS 'Classificação textual do risco de fogo';
COMMENT ON COLUMN {catalog}.{schema_silver}.{table_silver}.data_ref_carga IS 'Data de referência do processamento (AAAAMMDD)';
""")

print("📝 Descrições de colunas adicionadas com sucesso ✅")